In [1]:
## ####################################################
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score 

# load the data
X, y = load_breast_cancer(return_X_y=True)  ## X input, y output
print(X.shape, y.shape)

## to convert the {0,1} output into {-1,+1}
y = 2 * y -1

## X is the input matrix
mdata,ndim = X.shape
## normalization by L infinity norm
X/= np.outer(np.ones(mdata),np.max(np.abs(X),0))

## number of iteration 
niter = 10  

## penalty constant for the of the Stochastic Dual Coordinate Ascent algorithm
C = 1000

(569, 30) (569,)


In [ ]:
## ######## import sys
import sys
import numpy as np
import pylab as plt
from sklearn.datasets import load_breast_cancer
from sklearn.svm import SVC
from sklearn.metrics import f1_score, auc, roc_curve, roc_auc_score

def main(iworkmode):
    # load the data
    X, y = load_breast_cancer(return_X_y=True) ## X input, y output 
    print (X.shape, y.shape)
    ## to convert the {0,1} output into {-1,+1}
    y = 2 * y-1
    mdata, ndim = X.shape ## size of the data
    iscale = 1 ## =0 no scaling, =1 scaling the by the maximum absolute value 
    if iscale == 1: ## L_infty norm
        X = np.outer (np.ones (mdata), np. max (np. abs (X),0))
    elif iscale == 2: ## L2 norm
        xnorm = p.sqrt(np. sum (X**2,1)) 
        xnorm = xnorm + (xnorm==0)
        X = np.outer (xnorm, np.ones (ndim))
    
    ## number of iteration
    niter = 10
    ## set the penalty constant
    C = 1000
    ## dual stochastic gradient algorithm
    ## initialize alpha
    alpha = np.zeros(mdata)
    ## compute the linear kernel
    K = np.dot (X,X.T)
    for iter in range (niter):
        for i in range (mdata):
            ## process all sample examples sequentially
            ## sum on i!=j => sum on all i - (i=j)
            alpha[i] = (1-y[i]*(np.sum (K[i] *y*alpha)-alpha [i] *y [i]*K [i,i]))/K[i,i]
            ## projection
            alpha[i] = min (C/mdata, max (0, alpha[i]))
    ## accuracy of the full data
    Xtest = X
    mtest= X. shape [0]
    ## Linear kernel
    Kcross= np.dot (X, Xtest.T)
    ysdca = np.sign (np. sum (np.outer (y*alpha, np.ones (mtest)) *Kcross, 0))
    ## SVC predictions
    ## RBF
    svc_rbf = SVC(kernel = 'rbf', gamma ='scale')
    svc_rbf.fit(X,y) 
    ysvc_rbf = svc_rbf.predict(X)
    ## polynomial
    svc_poly = SVC(kernel = 'poly', degree= 5)
    svc_poly.fit(X,y)    
    ysvc_poly = svc_poly.predict(X)
    sdca_f1 = f1_score(y,ysdca)
    poly_f1 = f1_score(y,ysvc_poly)
    rbf_f1 = f1_score(y,ysvc_rbf)

    sdca_auc = roc_auc_score(y,ysdca)
    poly_auc = roc_auc_score(y,ysvc_poly)
    rbf_auc =roc_auc_score(y,ysvc_rbf)
    tsdca_roc = roc_curve(y,ysdca)
    sdca_auc2 = auc(tsdca_roc[0], tsdca_roc[1])
    tpoly_roc = roc_curve(y,ysvc_poly)
    poly_auc2= auc(tpoly_roc[0], tpoly_roc[1]) 
    trbf_roc = roc_curve (y, ysvc_rbf)
    rbf_auc2 = auc (trbf_roc[0], trbf_roc[1])
    print('Daul stochastic gradient F1:', '%5.3f'%sdca_f1)
    print('SVC Poly F1:','%5.3f'%poly_f1)
    print('SVC RBF F1:', '%5.3f'%rbf_f1)
    
    print('Daul stochastic gradient AUC:','%5.3f'%sdca_auc)
    print('SVC Poly AUC:', '%5.3f '%poly_auc)
    print('SVC RBF AUC:', '%5.3f'%rbf_auc)

    print('Daul stochastic gradient AUC2:','%5.3f'%sdca_auc2) 
    print('SVC Poly AUC2:','%5.3f'%poly_auc2)
    print('SVC RBF AUC2:','%5.3f'%rbf_auc2)

    plt.plot(tsdca_roc[0], tsdca_roc[1], 'r', tpoly_roc[0], tpoly_roc[1],'g', trbf_roc[0], trbf_roc [1], 'b')
    plt.show()
    print('Bye')
    return

# ## ######################
# ## ######################
# if __name__ == "__main__":
#     if len(sys.argv)==1:
#         iworkmode=0
#     elif len(sys.argv)>=2:
#         iworkmode=eval(sys.argv[1])
# main(iworkmode)